## Fitting to theory

Now that we have a rough idea of how the model behaves in simulation, we can fit our results to some theoretical predictions. I ran 140 dfferent simulations (for much longer than in the previous notebook) at 14 different values of $\beta$. Execute the cells below to pull up an interactive plot, and answer the questions.

In [ ]:
%load_ext cython
%matplotlib notebook


import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import glob
import h5py
from collections import defaultdict
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
from scipy.optimize import curve_fit

def magnetization(img):
    return np.sum(img)/np.prod(img.shape)

In [ ]:
%%cython
cimport cython
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def cy_ising_energy(np.int64_t[:, :] field):
    cdef int total = 0
    cdef int N = field.shape[0]
    cdef int M = field.shape[1]
    cdef int i, j
    cdef int n, m
    cdef float total_E = 0
    cdef float row_E = 0 
    for n in range(N):
        row_E = 0 
        for m in range(M):
            total = 0
            for i in range(n-1, n+2):
                for j in range(m-1, m+2):
                    if i == n and j == m:
                        continue
                    total += field[i % N, j % M]
            row_E += - field[n, m] * total
        total_E += row_E
    return total_E /(N * M)

In [ ]:
magnetization_by_temperature = defaultdict(list)
energy_by_temperature = defaultdict(list)
images_by_temperature = defaultdict(list)
for filename in glob.glob('run_*.hdf5'):
    with h5py.File(filename,'r') as input_file:
        beta = input_file.attrs['beta']
        img = np.array(input_file['image'])
        magnet = magnetization(img)
        energy = cy_ising_energy(img)
        magnetization_by_temperature[beta].append(magnet)
        energy_by_temperature[beta].append(energy)
        images_by_temperature[beta].append(img)

betas = np.array(sorted(list(set(magnetization_by_temperature.keys()))))
e_average = np.array([np.mean(energy_by_temperature[beta]) for beta in betas])
e_sigma = np.array([np.std(energy_by_temperature[beta]) for beta in betas])/np.sqrt(10)

In [ ]:
def show_data():
    gs = GridSpec(ncols=5, nrows=25)
    betas = magnetization_by_temperature.keys()
    all_energies = []
    all_magnets = []
    all_betas = []
    for b in betas:
        all_energies.extend(energy_by_temperature[b])
        all_magnets.extend(magnetization_by_temperature[b])
        all_betas.extend([float(b)]*10)
    
    fig = plt.figure(figsize = (9, 9))
    img_axes = []
    mag_ax = fig.add_subplot(gs[0:9, :])
    e_ax = fig.add_subplot(gs[11:17, :])
    for i in range(2):
        for j in range(5):
            ax = fig.add_subplot(gs[19 + 3*i : 22 + 3*i, j])
            img_axes.append((i, j, ax))
    
    def show_beta(beta):
        beta = float(beta)
        mag_ax.clear()
        mag_ax.plot(all_betas, all_magnets, 'ok', alpha= 0.3)   
        mag_ax.plot([beta] * 10, magnetization_by_temperature[beta], 'or', alpha= 1.0)
        mag_ax.set_xlabel(r"$\beta$")
        mag_ax.set_ylabel("Magnetization")
        mag_ax.set_title("beta = {:0.02f}".format(beta))

        e_ax.clear()
        e_ax.plot(all_betas, all_energies, 'ok', alpha= 0.3)
        e_ax.plot([beta] * 10, energy_by_temperature[beta], 'or', alpha= 1.0)

        e_ax.set_xlabel(r"$\beta$")
        e_ax.set_ylabel("Energy")

        for i, j, ax in img_axes:
            ax.clear()
            ax.imshow(1 - images_by_temperature[beta][5*i + j], 'gray')
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)
        plt.show()
    
    beta_selector = widgets.SelectionSlider(
        options=['{:0.2f}'.format(b) for b in sorted(set(all_betas))],
        description='beta = ',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        layout=widgets.Layout(width='90%')
    )
    interact(show_beta, beta=beta_selector)

## Explore the phase diagram

First, let's explore the phase diagram of the Ising model. Describe what happens as you increase the temperature.

What's going on with the data at $\beta = 0.26$?

Enter your answer here.

In [ ]:
show_data()

One simplifed treament suggests that the energy $U$ of the Ising model has the following functional dependence on $\beta$

$$ U = n \left(\frac{1}{\exp\left(\alpha(\beta-\beta_c)\right) + 1} - 1\right) $$

where n is the number of neighbor is the Ising model ($n=8$ in 2D), and $\alpha$ and $\beta_c$ are free parameters. Use `scipy.curve_fit` to find them.

The variables defined in this notebook that you will need are

`betas` = the $\beta$s at which the energy was measured

`e_average` = the average energy $U$ at each beta


`e_sigma` = the standard error in energy $U$ at each beta


In [ ]:
def e_theory(beta, alpha, beta_c):
    return 8 * (1 / (np.exp(alpha*(beta-beta_c)) + 1) - 1)

params, _ = curve_fit(e_theory, betas, e_average, sigma=e_sigma)

print('alpha: {}, beta_c: {}'.format(params[0], params[1]))

Now, plot it below.

In [ ]:
plt.figure()
beta_theory = np.linspace(0.1, 0.31, 1000)
plt.plot(beta_theory, e_theory(beta_theory, *params), '-k', label = 'fit')
plt.errorbar(betas, e_average, yerr=3*e_sigma, fmt='.r', label='data')
plt.ylabel('Energy')
plt.xlabel(r'$\beta$')
plt.show()

How well does this model fit our data? Support your answer.

Answer here